In [ ]:
#sentence alignment
#theory behind id word level levenshtein distance
#! /bin/python
#-*- coding:utf-8 -*-

import numpy
class Alignment():
    def __init__(self):
        self.ar = []
        self.ah = []            
        self.numSub = 0;
        self.numDel = 0;
        self.numIns = 0;
        self.rstring = ""
        self.hstring = ""
    
    def align(self,h,r):
        OK = 0
        SUB = 1
        INS = 2
        DEL = 3
    
        h = h.split(" ")
        r = r.split(" ")
        cost = numpy.zeros((len(r)+1)*(len(h)+1), dtype=numpy.uint8)
        cost = cost.reshape((len(r)+1, len(h)+1))
    
        backtrace = numpy.zeros((len(r)+1)*(len(h)+1), dtype=numpy.uint8)
        backtrace = backtrace.reshape((len(r)+1, len(h)+1))
    
        for i in range(len(r)+1):
            for j in range(len(h)+1):
                if i == 0:
                    cost[0][j] = j
                    backtrace[0][j] = INS
                elif j == 0:
                    cost[i][0] = i
                    backtrace[i][0] = DEL    
        
        for i in range(1,len(r)+1):
            for j in range(1,len(h)+1):
                subOp = ""
                cs = ""
                if r[i-1] == h[j-1]:
                    subOp = OK
                    cs = cost[i-1][j-1]
                else:
                    subOp = SUB
                    cs = cost[i-1][j-1]+1
                ci = cost[i][j-1] +1
                cd = cost[i-1][j] +1
                mincost = min(cs, min(ci, cd));
                if cs == mincost:
                    cost[i][j] = cs;
                    backtrace[i][j] = subOp;
                elif ci == mincost:
                    cost[i][j] = ci;
                    backtrace[i][j] = INS;                    
                else :
                    cost[i][j] = cd;
                    backtrace[i][j] = DEL; 
        
        i = len(r);
        j = len(h);            
        while i > 0 or j > 0:#     treeinfo2 = Treeinfo()
#     print treeinfo2.get_nodes(rtree)
#     print treeinfo2.ordered_pos(rtree)
            if backtrace[i][j] == OK:
                self.ar.insert(0,r[i-1].lower())
                self.ah.insert(0,h[j-1].lower())
                i = i-1
                j = j-1
                
            elif backtrace[i][j] == SUB:
                self.ar.insert(0,r[i-1].upper()+">SUB")
                self.ah.insert(0,h[j-1].upper()+">SUB")
                i = i-1
                j = j-1
                self.numSub = self.numSub+1
                
            elif backtrace[i][j] == INS:
                self.ar.insert(0,"NONE")
                self.ah.insert(0,h[j-1]+">INS")
                j = j-1
                self.numIns = self.numIns+1
                
            elif backtrace[i][j] == DEL:
                self.ar.insert(0,r[i-1].upper()+">DEL")
                self.ah.insert(0,"NONE")
                i = i-1
                self.numDel = self.numDel+1
        self.rstring = " ".join(self.ar)
#         for a in self.ah:
#             print a
        self.hstring = " ".join(self.ah)

def testalign():
   
    tl = "This is the surprise"
    rl = "this is a big surprise"
    alignment = Alignment()
    alignment.align(tl,rl)
    print alignment.ah
    print alignment.ar
    print "Substitution: ",alignment.numSub,"Insertion: ",alignment.numIns,"Deletion: ",alignment.numDel
                    
if __name__ == "__main__":
    testalign()

In [ ]:
#use stanford parser
#! /bin/python
#-*- coding:utf-8 -*-

import os
from nltk.parse import stanford
from nltk.tree import Tree
from time import time

#Stanford Parser is conposed of constituent parser("con"),dependency parser("de")and nueral parser("neu")
#language choices are Chinese,English,French,German,and Spanish
class Parser():
    def __init__(self):
        self.parser = ""
    def setlang(self,lang,kind):
        os.environ['STANFORD_PARSER'] = '/home/bear/Downloads/parser/stanford-parser-full-2015-12-09/'
        os.environ['JAVA_HOME'] = '/usr/lib/jvm/jdk1.8.0_91/jre/jre/bin/'
        os.environ['STANFORD_MODELS'] = '/home/bear/Downloads/parser/stanford-parser-full-2015-12-09/stanford-parser-3.6.0-models.jar'
        if lang == "English":
            if kind == "con":
                self.parser = stanford.StanfordParser(model_path="/home/bear/Downloads/parser/stanford-parser-full-2015-12-09/stanford-parser-3.6.0-models/edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz")
            elif kind == "de":
                pass
            elif kind == "neu":
                pass
        elif lang == "Chinese":
            if kind == "con":
                self.parser = stanford.StanfordParser(model_path="/home/bear/Downloads/parser/stanford-parser-full-2015-12-09/stanford-parser-3.6.0-models/edu/stanford/nlp/models/lexparser/chineseFactored.ser.gz")
            elif kind == "de":
                pass
            elif kind == "neu":
                pass
        elif lang == "French":
            if kind == "con":
                self.parser = stanford.StanfordParser(model_path="/home/bear/Downloads/parser/stanford-parser-full-2015-12-09/stanford-parser-3.6.0-models/edu/stanford/nlp/models/lexparser/frenchFactored.ser.gz")
            elif kind == "de":
                pass
            elif kind == "neu":
                pass
        elif lang == "German":
            if kind == "con":
                self.parser = stanford.StanfordParser(model_path="/home/bear/Downloads/parser/stanford-parser-full-2015-12-09/stanford-parser-3.6.0-models/edu/stanford/nlp/models/lexparser/germanPCFG.ser.gz")
            elif kind == "de":
                pass
            elif kind == "neu":
                pass
        elif lang == "Spanish":
            if kind == "con":
                self.parser = stanford.StanfordParser(model_path="/home/bear/Downloads/parser/stanford-parser-full-2015-12-09/stanford-parser-3.6.0-models/edu/stanford/nlp/models/lexparser/spanishPCFG.ser.gz")
            elif kind == "de":
                pass
            elif kind == "neu":
                pass  
               

In [ ]:
#extract parsed tree
#! /bin/python
#-*- coding:utf-8 -*-

import nltk

class Treeinfo():
    def __init__(self):   
        self.treelabel = []
        self.pos = []
        self.ROOT = 'ROOT'
        self.orderedpos = []
        self.word = []
    def get_nodes(self,parent):

        for node in parent:
            if type(node) is nltk.Tree:
                if node.height() > 2:
                    self.treelabel.append(node.label())
                    #print "Label:", node.label()
                elif node.height() == 2:
                    self.pos.append((node.label()))
                    #print node.leaves
                    #print node.label()

                self.get_nodes(node)
            else:
                self.word.append(node)
    
                
    def ordered_pos(self,wordlist):
        for w in wordlist:
            for p in self.pos:
                if w == "".join(p[1]):
                    self.orderedpos.append(p[0])
        #print len(self.orderedpos)

if __name__ == "__main__":
    pass

In [ ]:
#fscore
#! /bin/python
#-*- coding:utf-8 -*-

import numpy

def wer(h,r):
    d = numpy.zeros((len(h)+1)*(len(r)+1), dtype=numpy.uint8)
    d = d.reshape((len(h)+1, len(r)+1))
    for i in range(len(h)+1):
        for j in range(len(r)+1):
            if i == 0:
                d[0][j] = j
            elif j == 0:
                d[i][0] = i

    # computation
    for i in range(1, len(h)+1):
        for j in range(1, len(r)+1):
            if h[i-1] == r[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                substitution = d[i-1][j-1] + 1
                insertion    = d[i][j-1] + 1
                deletion     = d[i-1][j] + 1
                d[i][j] = min(substitution, insertion, deletion)
                

    return d[len(h)][len(r)]
class Alignment():
    def __init__(self):
        self.ar = []
        self.ah = []            
        self.numSub = 0;
        self.numDel = 0;
        self.numIns = 0;
    
    def align(self,h,r):
        OK = 0
        SUB = 1
        INS = 2
        DEL = 3
    
        h = h.split(" ")
        r = r.split(" ")
        cost = numpy.zeros((len(r)+1)*(len(h)+1), dtype=numpy.uint8)
        cost = cost.reshape((len(r)+1, len(h)+1))
    
        backtrace = numpy.zeros((len(r)+1)*(len(h)+1), dtype=numpy.uint8)
        backtrace = backtrace.reshape((len(r)+1, len(h)+1))
    
        for i in range(len(r)+1):
            for j in range(len(h)+1):
                if i == 0:
                    cost[0][j] = j
                    backtrace[0][j] = INS
                elif j == 0:
                    cost[i][0] = i
                    backtrace[i][0] = DEL    
        
        for i in range(1,len(r)+1):
            for j in range(1,len(h)+1):
                subOp = ""
                cs = ""
                if r[i-1] == h[j-1]:
                    subOp = OK
                    cs = cost[i-1][j-1]
                else:
                    subOp = SUB
                    cs = cost[i-1][j-1]+1
                ci = cost[i][j-1] +1
                cd = cost[i-1][j] +1
                mincost = min(cs, min(ci, cd));
                if cs == mincost:
                    cost[i][j] = cs;
                    backtrace[i][j] = subOp;
                elif ci == mincost:
                    cost[i][j] = ci;
                    backtrace[i][j] = INS;                    
                else :
                    cost[i][j] = cd;
                    backtrace[i][j] = DEL; 
        
        i = len(r);
        j = len(h);            
        while i > 0 or j > 0:
            if backtrace[i][j] == OK:
                self.ar.insert(0,r[i-1].lower())
                self.ah.insert(0,h[j-1].lower())
                i = i-1
                j = j-1
                
            elif backtrace[i][j] == SUB:
                self.ar.insert(0,r[i-1].upper())
                self.ah.insert(0,h[j-1].upper())
                i = i-1
                j = j-1
                self.numSub = self.numSub+1
                
            elif backtrace[i][j] == INS:
                self.ar.insesrt(0,None)
                self.ah.insert(0,h[j-1])
                j = j-1
                self.numIns = self.numIns+1
                
            elif backtrace[i][j] == DEL:
                self.ar.insert(0,r[i-1].upper())
                self.ah.insert(0,None)
                i = i-1
                self.numDel = self.numDel+1

def fscore (test,ref):

    tp = 0.0
    for t in test:
        if t in ref:
            tp +=1
    
    precision = tp/len(ref)
    recall = tp/len(test)
    fscore = 2*precision*recall/(precision+recall)
        
    return fscore

def weighted_average(alpha,beta,gamma,tree_score,pos_score,vacabulary_score):
    score = (alpha*tree_score+beta*pos_score+gamma*vacabulary_score)/(alpha+beta+gamma)
    return score
    
if __name__ == "__main__":
    print weighted_average(10, 3, 4, 0.8, 0.3, 0.8)

In [ ]:
#! /bin/python
#-*- coding:utf-8 -*

import numpy

def wer(r, h):
    #build the matrix
    d = numpy.zeros((len(r)+1)*(len(h)+1), dtype=numpy.uint8).reshape((len(r)+1, len(h)+1))
    for i in range(len(r)+1):
        for j in range(len(h)+1):
            if i == 0: 
                d[0][j] = j
            elif j == 0:
                d[i][0] = i
    for i in range(1,len(r)+1):
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                d[i][j] = d[i-1][j-1]
            else:
                substitute = d[i-1][j-1] + 1
                insert = d[i][j-1] + 1
                delete = d[i-1][j] + 1
                d[i][j] = min(substitute, insert, delete)

    result = 1-float(d[len(r)][len(h)]) / len(r) 
    return result                  

def fscore (test,ref):
    common = []
    for i in test:
            if i in ref:
                common.append(i)
                ref.remove(i)
    
    tp = len(common)
    fp = len(test) - tp 
    fn = len(ref)
    if tp>0:
        precision=float(tp)/(tp+fp)
        recall=float(tp)/(tp+fn)
        return 2*((precision*recall)/(precision+recall))
    else:
        return 0
        

def weighted_average(alpha,beta,gamma,tree_score,pos_score,vocabulary_score):
    score = (alpha*tree_score+beta*pos_score+gamma*vocabulary_score)/(alpha+beta+gamma)
    return score

In [ ]:
#! /bin/python
#-*- coding:utf-8 -*-

import parse
import alignment
import caculation
import extract_tree
import codecs
from nltk import Tree

#use stanford parser
def get_tree(sen,lang,kind):
    p = parse.Parser()
    p.setlang(lang, kind)
    tree = p.parser.raw_parse(sen.lower())
    return tree

#extract labels and leaves
def get_treeinfo(tree):
    t = extract_tree.Treeinfo()
    t.get_nodes(tree)
    return t.treelabel,t.pos,t.word



def score_similarity(htree,rtree): 

    hlabel,hpos,hword = get_treeinfo(htree)
    rlabel,rpos,rword = get_treeinfo(rtree)

    tree_score = caculation.fscore(hlabel, rlabel)
    pos_score = caculation.wer(hpos, rpos)
    vocabulary_score = caculation.fscore(hword,rword)
    score = caculation.weighted_average(0.35, 0.35, 0.3, tree_score, pos_score, vocabulary_score)  
    return score,hword,rword

def print_alignment(hyp,ref):
    a = alignment.Alignment()
    a.align(hyp,ref) #input is string!!!
    print "Formatted hypothesis: ", a.hstring
    print "Formatted reference: ", a.rstring
    print "Substitution: ",a.numSub,"Insertion: ",a.numIns,"Deletion: ",a.numDel
    

def read_treefile(hyptreefile,reftreefile):
    hfile = codecs.open(hyptreefile,"r",encoding='utf-8')
    rfile = codecs.open(reftreefile,"r",encoding='utf-8')
    scoredic = {}
    #store rtree into rtreelist suppose there are more than one reference
    rtreel = []
    for i in rfile:
        refl = []
        if i.strip() != "":
            refl.append(i.strip())
            rstr = " ".join(refl)
            rtree = Tree.fromstring(rstr)
        rtreel.append(rtree)
    #store hyptree into hyplist    
    htreel = []
    senl = []
    for i in hfile:
        if i.strip() != "":
            senl.append(i.strip())
        else:
            htreel.append(Tree.fromstring(" ".join(senl)))
            senl = []
            
    #loop and score
    for r in rtreel:
        for h in htreel:
            score,hword,rword= score_similarity(h,r)
            scoredic[" ".join(hword)] = score
            
    return scoredic     

    
def sort_scoredic(scoredic):
    order = sorted(scoredic, key=scoredic.get, reverse=True)
    #print "score time:", round(time()-t0, 3), "s"            
    for i in range(15):
        if i < len(order):
            print "key:", order[i],"value: ", scoredic[order[i]] 
                            

def read_rawfile(hyprawfile,refrawfile):
    scoredic = {}
    cnt = 0
    for i in refrawfile:
        for j in hyprawfile:
            cnt += 1
            print cnt
            hyp = j.strip()
            ref = i.strip()
            
            htree = get_tree(hyp, "English","con") #constituent parser
            rtree = get_tree(ref, "English","con")
            
            score,hword,rword = score_similarity(htree,rtree)
            scoredic[" ".join(hword)] = score
            
    return scoredic
    
if __name__ == "__main__":  
    #single sentence test
#     hyp = "The stock posted a multi-year low of under $10 in early 2009 before roaring to a recent high of nearly $75 in April 2014."
#     ref = "Before roaring to a recent high of nearly $75 in April 2014 the stock posted a multi-year low of under $10 in early 2009."
# 
#     htree = get_tree(hyp, "English","con") #constituent parser
#     rtree = get_tree(ref, "English","con")
# 
#     score,hword,rwprd = score_similarity(htree, rtree)
#     print score
#     print_alignment(hyp, ref)

    #rawfile test
#     hyprawfile = codecs.open("/home/work/Data/similartest/smallentest.txt","r",encoding='utf-8')
#     refrawfile = codecs.open("/home/work/Data/similartest/zhref.txt","r",encoding='utf-8')
#     scoredic = read_rawfile(hyprawfile, refrawfile)
#     sort_scoredic(scoredic)
    
    #treefile test
    hyptreefile = "/home/work/Data/similartest/twozhtest.parse"
    reftreefile = "/home/work/Data/similartest/onezhref.parse"
    scoredic = read_treefile(hyptreefile, reftreefile)
    sort_scoredic(scoredic)
    